<a href="https://colab.research.google.com/github/LucasAguiar00/Trabalho_TopicosII/blob/main/Problema_da_%C3%81rvore_Geradora_M%C3%ADnima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema da Árvore Geradora Mínima

## Bibliotecas utilizadas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import heapq

import time
import os
import psutil
import random

import tracemalloc

In [ ]:
pip install psutil

In [ ]:
# Função para calcular o uso da memória
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss  # Em bytes
    # return process.memory_info().rss / 1024 / 1024  # Em megabytes

In [ ]:
# Monta o Google Drive no Colab para ler a instâncias no drive, os arquivos ficaram dispostos com base nas pastas abaixo
  # pasta = '/content/drive/My Drive/AGM/ALUE'
  # pasta = '/content/drive/My Drive/AGM/ALUT'
  # pasta = '/content/drive/My Drive/AGM/DMXA'
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Funções para a leitura do GRAFO

In [ ]:
# Lê o grafo com base no caminho do arquivo e retorna o array
def lerGrafo(nome_arquivo):
    dados = []
    with open(nome_arquivo, 'r') as arquivo:
        for linha in arquivo:
            if linha.startswith('E'):
                valores = linha.split()[1:]  # Ignorar o 'E' no início
                if len(valores) == 3:
                    x, y, z = map(float, valores)
                    dados.append([x, y, z])
    return dados

def gerarGrafo(nome_arquivo):
  matriz_dados = lerGrafo(nome_arquivo)
  matriz_dados = np.round(matriz_dados).astype(int) # Mudando os dados de float para inteiro
  matriz_dados = matriz_dados.tolist() # Mudando o formato de np.array para list
  return matriz_dados

# # Exemplo de uso
# pasta = '/content/drive/My Drive/AGM/DMXA'
# instância = 'dmxa0368.stp'
# caminho = f'{pasta}/{instância}'
# matriz_dados = gerarGrafo(caminho)
# print(matriz_dados)

In [ ]:
# Lista as instâncias que estão dentro de cada pasta dos grafos disponibilizados
def listar_arquivos_em_pasta(caminho_pasta):
    try:
        # Verifica se o caminho é uma pasta
        if os.path.isdir(caminho_pasta):
            # Lista os arquivos na pasta
            arquivos = os.listdir(caminho_pasta)
            return arquivos
        else:
            return None
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

# # Exemplo de uso
# pasta = '/content/drive/My Drive/AGM/DMXA'
# arquivos_na_pasta = listar_arquivos_em_pasta(pasta)
# print(arquivos_na_pasta)

## B) Implementação da AVL

In [ ]:
# AVL
class AVLTree:
    class Node:
        def __init__(self, key):
            self.key = key
            self.left = None
            self.right = None
            self.height = 1

    def __init__(self):
        self.root = None

    def height(self, node):
        if not node:
            return 0
        return node.height

    def update_height(self, node):
        if not node:
            return 0
        node.height = 1 + max(self.height(node.left), self.height(node.right))

    def balance_factor(self, node):
        if not node:
            return 0
        return self.height(node.left) - self.height(node.right)

    def rotate_left(self, y):
        x = y.right
        T2 = x.left

        x.left = y
        y.right = T2

        y.height = 1 + max(self.height(y.left), self.height(y.right))
        x.height = 1 + max(self.height(x.left), self.height(x.right))

        return x

    def rotate_right(self, x):
        y = x.left
        T2 = y.right

        y.right = x
        x.left = T2

        x.height = 1 + max(self.height(x.left), self.height(x.right))
        y.height = 1 + max(self.height(y.left), self.height(y.right))

        return y

    def insert(self, node, key):
        if not node:
            return self.Node(key)

        if key < node.key:
            node.left = self.insert(node.left, key)
        else:
            node.right = self.insert(node.right, key)

        self.update_height(node)

        balance = self.balance_factor(node)

        # Caso de desequilíbrio à esquerda
        if balance > 1:
            if key < node.left.key:
                return self.rotate_right(node)
            else:
                node.left = self.rotate_left(node.left)
                return self.rotate_right(node)

        # Caso de desequilíbrio à direita
        if balance < -1:
            if key > node.right.key:
                return self.rotate_left(node)
            else:
                node.right = self.rotate_right(node.right)
                return self.rotate_left(node)

        return node

In [ ]:
## Função PRIM com AVL
def prim_mst(matrix):
    if not matrix:
        return None

    n = len(matrix)
    min_heap = []  # Heap mínimo para escolher as arestas de menor peso
    visited = [False] * n
    mst = []  # Lista para armazenar as arestas da árvore geradora mínima

    # Crie um grafo vazio
    G = nx.Graph()

    # Inicie com o primeiro vértice
    start_vertex = matrix[0][0]
    visited[start_vertex] = True

    for i in range(n - 1):
        for edge in matrix:
            u, v, weight = edge
            if visited[u] != visited[v]:
                heapq.heappush(min_heap, (weight, edge))

        while min_heap:
            weight, edge = heapq.heappop(min_heap)
            u, v, _ = edge
            if visited[u] != visited[v]:
                visited[u] = True
                visited[v] = True
                mst.append(edge)
                G.add_edge(u, v, weight=weight)
                break

    return G, mst

# # Exemplo de uso
# matrix = [
#     [0, 1, 2],
#     [0, 2, 4],
#     [1, 2, 3],
#     [1, 3, 1],
#     [2, 3, 5],
#     [0, 4, 6],
#     [2, 4, 2],
#     [3, 4, 8]
# ]

# G, mst = prim_mst(matrix)

# # Desenhe o grafo resultante
# pos = nx.spring_layout(G)
# labels = nx.get_edge_attributes(G, 'weight')
# nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_size=10)
# nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
# plt.title("Árvore Geradora Mínima (Prim)")
# plt.show()

## Executando a AVL

In [ ]:
## OBSERVAÇÃO:: NECESSÁRIO MUDAR A PASTA MANUALMENTE PARA PROCESSAR OS MODELOS DE INSTÂNCIA ALUE | ALUT E DMXA

# pasta = '/content/drive/My Drive/AGM/ALUE'
pasta = '/content/drive/My Drive/AGM/ALUT'
# pasta = '/content/drive/My Drive/AGM/DMXA'

# Lista todas as instâncias que estão na pasta
todasInstâncias = listar_arquivos_em_pasta(pasta)

# Executa cada instância 10x exibindo o tempo e memória usadas
for instância in todasInstâncias:
  caminho = f'{pasta}/{instância}'
  matriz_dados = gerarGrafo(caminho)

  res_tempo = [];
  res_memória = [];
  for i in range(10):
    # Registra a memória usada inicialmente e o tempo inicial
    inicio_memoria = psutil.virtual_memory()

    inicio = time.time()

    # Chamando a função
    G, mst = prim_mst(matriz_dados)

    # Registra a memória usada após executar a função e o tempo final
    fim = time.time()
    fim_memoria = psutil.virtual_memory()

    # Realiza o cálculo do tempo decorrido e do uso da memória
    tempo_decorrido = fim - inicio
    uso_memoria = abs(fim_memoria.used - inicio_memoria.used)*8 # x 8 para ficar em bits

    # Adicionando o resultado nos vetors
    res_tempo.append(tempo_decorrido)
    res_memória.append(uso_memoria)

  print(f'Instância {instância}')
  print(f'Tempo médio (s): {np.mean(res_tempo)}')
  print(f'Tempo máximo (s): {np.max(res_tempo)}')
  print(f'Tempo minimo (s): {np.min(res_tempo)}')
  print(f'Memória médio (bits): {np.mean(res_memória)}')
  print(f'Memória máximo (bits): {np.max(res_memória)}')
  print(f'Memória minimo (bits): {np.min(res_memória)}')
  print('\n')

Instância alue6179.stp
Tempo médio (s): 4.041339588165283
Tempo máximo (s): 9.2859628200531
Tempo minimo (s): 2.6262638568878174
Memória médio (bits): 371248332.8
Memória máximo (bits): 1740603392
Memória minimo (bits): 0


Instância alue7229.stp
Tempo médio (s): 0.39571051597595214
Tempo máximo (s): 0.4711153507232666
Tempo minimo (s): 0.21811866760253906
Memória médio (bits): 1726873.6
Memória máximo (bits): 10321920
Memória minimo (bits): 0


Instância alue3146.stp
Tempo médio (s): 4.681695365905762
Tempo máximo (s): 5.941761016845703
Tempo minimo (s): 3.721456527709961
Memória médio (bits): 80140697.6
Memória máximo (bits): 572915712
Memória minimo (bits): 65536


Instância alue7066.stp
Tempo médio (s): 14.26295177936554
Tempo máximo (s): 14.798033237457275
Tempo minimo (s): 14.015328407287598
Memória médio (bits): 84384153.6
Memória máximo (bits): 341999616
Memória minimo (bits): 5373952


Instância alue2087.stp
Tempo médio (s): 0.3934450387954712
Tempo máximo (s): 0.4609622955322

## C) Implementação Heap Fibonacci

In [ ]:
pip install heapdict

In [ ]:
pip install networkx matplotlib

In [ ]:
import heapdict
import networkx as nx
import matplotlib.pyplot as plt

def prim_fibonacci_heap(matrix):
    # Crie um dicionário para representar o grafo com base na matriz de entrada
    graph = {}

    # Preencha o dicionário com vértices e arestas
    for row in matrix:
        u, v, weight = row
        if u not in graph:
            graph[u] = []
        if v not in graph:
            graph[v] = []
        graph[u].append((v, weight))
        graph[v].append((u, weight))

    # Inicialize a estrutura de dados Heap de Fibonacci e o conjunto de vértices visitados
    fib_heap = heapdict.heapdict()
    visited = set()

    # Escolha um vértice arbitrário para iniciar
    start_vertex = list(graph.keys())[0]

    # Inicie a AGM com o vértice inicial
    agm = []
    visited.add(start_vertex)

    # Adicione todas as arestas conectadas ao vértice inicial à Heap de Fibonacci
    for neighbor, weight in graph[start_vertex]:
        fib_heap[neighbor] = (start_vertex, neighbor, weight)

    # Continue até que todos os vértices sejam visitados
    while len(visited) < len(graph):
        # Encontre a aresta com o menor peso para a AGM
        _, current_edge = fib_heap.popitem()

        # Desempacote a aresta
        u, v, min_weight = current_edge

        # Adicione o vértice e a aresta à AGM
        agm.append((u, v, min_weight))
        visited.add(v)

        # Atualize a Heap de Fibonacci com as arestas dos vértices recém-adicionados
        for neighbor, weight in graph[v]:
            if neighbor not in visited:
                # Se o vértice ainda não foi visitado, verifique se a aresta é menor que a atual na Heap de Fibonacci
                if neighbor not in fib_heap or weight < fib_heap[neighbor][2]:
                    fib_heap[neighbor] = (v, neighbor, weight)

    return agm

# Exemplo de uso:
# matrix = [
#     ('A', 'B', 2),
#     ('A', 'C', 2),
#     ('B', 'C', 1),
#     ('B', 'D', 1),
#     ('C', 'D', 5),
#     ('E', 'B', 1),
#     ('E', 'C', 1),
# ]

# agm = prim_fibonacci_heap(matrix)

# # Crie um grafo para visualização
# G = nx.Graph()
# G.add_weighted_edges_from([(u, v, weight) for u, v, weight in agm])

# # Plote o grafo
# pos = nx.spring_layout(G)
# labels = nx.get_edge_attributes(G, 'weight')
# nx.draw(G, pos, with_labels=True, node_size=500, node_color='lightblue')
# nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
# plt.title("Árvore Geradora Mínima")
# plt.show()


## Executando o Heap Fibonacci

In [ ]:
## OBSERVAÇÃO:: NECESSÁRIO MUDAR A PASTA MANUALMENTE PARA PROCESSAR OS MODELOS DE INSTÂNCIA ALUE | ALUT E DMXA

pasta = '/content/drive/My Drive/AGM/ALUE'
# pasta = '/content/drive/My Drive/AGM/ALUT'
# pasta = '/content/drive/My Drive/AGM/DMXA'

# Lista todas as instâncias que estão na pasta
todasInstâncias = listar_arquivos_em_pasta(pasta)

# Executa cada instância 10x exibindo o tempo e memória usadas
for instância in todasInstâncias:
  caminho = f'{pasta}/{instância}'
  matriz_dados = gerarGrafo(caminho)

  res_tempo = [];
  res_memória = [];
  for i in range(10):
    # Registra a memória usada inicialmente e o tempo inicial
    inicio_memoria = psutil.virtual_memory()
    inicio = time.time()

    # Chamando a função
    agm = prim_fibonacci_heap(matriz_dados)

    # Registra a memória usada após executar a função e o tempo final
    fim = time.time()
    fim_memoria = psutil.virtual_memory()

    # Realiza o cálculo do tempo decorrido e do uso da memória
    tempo_decorrido = fim - inicio
    uso_memoria = abs(fim_memoria.used - inicio_memoria.used)*8 # x 8 para ficar em bits

    # Adicionando o resultado nos vetors
    res_tempo.append(tempo_decorrido)
    res_memória.append(uso_memoria)

  print(f'Instância: {instância}')
  print(f'Tempo médio (s): {np.mean(res_tempo)}')
  print(f'Tempo máximo (s): {np.max(res_tempo)}')
  print(f'Tempo minimo (s): {np.min(res_tempo)}')
  print(f'Memória médio (bits): {np.mean(res_memória)}')
  print(f'Memória máximo (bits): {np.max(res_memória)}')
  print(f'Memória minimo (bits): {np.min(res_memória)}')
  print('\n')